In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.6.3
sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.21.6
pandas 1.3.5
sklearn 1.0.2
tensorflow 2.6.3
keras.api._v2.keras 2.6.0


In [2]:
# https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
input_filepath = "../input/shakespeare/shakespeare.txt"
text = open(input_filepath, 'r').read()

print(len(text))
print(text[0:100])

1115393
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
# 1. generate vocab
# 2. build mapping char->id
# 3. data -> id_data  把数据都转为id
# 4. abcd -> bcd<eos>  预测下一个字符生成的模型，也就是输入是a，输出就是b

# 去重，留下独立字符，并排序
vocab = sorted(set(text))
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
# 体会enumerate接口的作用
for idx,char in enumerate(['how','are','you']):
    print(idx,char)
for i in enumerate(['how','are','you']):
    print(i)
print(type(enumerate(['how','are','you'])))

0 how
1 are
2 you
(0, 'how')
(1, 'are')
(2, 'you')
<class 'enumerate'>


In [5]:
# enumerate对每一个位置处字符编号
# 迭代器enumerate，其中每个元素都是元组(,)，下面是字典生成式
char2idx = {char:idx for idx, char in enumerate(vocab)}
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [6]:
# 把vocab从列表变为ndarray
idx2char = np.array(vocab)
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [7]:
# 把字符都转换为id
text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int.shape)
print(len(text_as_int))
print(text_as_int[0:10])
print(text[0:10])

(1115393,)
1115393
[18 47 56 57 58  1 15 47 58 47]
First Citi


In [8]:
# 把输入和输出分配好（细操）
def split_input_target(id_text):
    """
    abcde -> abcd, bcde,输入是abcd，输出是bcde
    """
    return id_text[0:-1], id_text[1:]

# 把id_text转换为 dataset
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
# 做一个batch，seq_length + 1目的是我们输入是5个字符时，输出是4
# drop_remainder是最后不够就丢掉，这个batch是把字变为句子，一个句子是101个字符
seq_dataset = char_dataset.batch(seq_length + 1,
                                 drop_remainder = True)
for ch_id in char_dataset.take(2):
    print(ch_id, idx2char[ch_id.numpy()])

# seq_dataset 每一个都是句子（id组成）
for seq_id in seq_dataset.take(2):
    print(seq_id)
    print(repr(''.join(idx2char[seq_id.numpy()])))

tf.Tensor(18, shape=(), dtype=int64) F
tf.Tensor(47, shape=(), dtype=int64) i
tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int64)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int64)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'


2022-05-19 13:41:10.911557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 13:41:11.036119: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 13:41:11.036885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 13:41:11.042923: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [9]:
# 然后通过split_input_target函数来对seq_dataset做映射，得到输入，输出
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input)
    print(item_output)
print(seq_dataset)

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59], shape=(100,), dtype=int64)
tf.Tensor(
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1], shape=(100,), dtype=int64)
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 

2022-05-19 13:41:13.781996: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [10]:
batch_size = 64
buffer_size = 10000
# 这个batch是真正的batch，上一个batch是把字变为句子，buffer_size是从数据集拿多少元素
seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)
print(seq_dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [11]:
vocab_size = len(vocab)
embedding_dim = 256  # 资料比较小，所以dim可以设大一些
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim,
                               batch_input_shape = [batch_size, None]),
        # return_sequences是指要返回一个序列，也就是所有输出，而不是最后一个
        keras.layers.SimpleRNN(units = rnn_units,
                               stateful = True,  # 是否把上一批最后返回的状态添加到下一批作为输入
                               recurrent_initializer = 'glorot_uniform',
                               return_sequences = True),
        # 全连接层，为什么最后一层全连接层的输出是vocab_size
        keras.layers.Dense(vocab_size),
    ])
    return model

model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = batch_size)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
simple_rnn (SimpleRNN)       (64, None, 1024)          1311744   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.variables

[<tf.Variable 'embedding/embeddings:0' shape=(65, 256) dtype=float32, numpy=
 array([[ 0.01919475, -0.01261197,  0.0445695 , ..., -0.02104477,
         -0.0435562 , -0.00419811],
        [-0.03338762, -0.01135081, -0.02068409, ...,  0.03816238,
          0.0497164 ,  0.02503288],
        [ 0.03240658, -0.01093385, -0.0321333 , ...,  0.00417049,
         -0.03226669, -0.01414679],
        ...,
        [-0.0222993 ,  0.01016422, -0.00263877, ..., -0.02878145,
         -0.04438659,  0.00964663],
        [ 0.04537973, -0.02534884,  0.02271712, ..., -0.03358071,
          0.02241695,  0.01282773],
        [ 0.04327709, -0.02299103, -0.00440223, ..., -0.03154824,
          0.02054248,  0.0226951 ]], dtype=float32)>,
 <tf.Variable 'simple_rnn/simple_rnn_cell/kernel:0' shape=(256, 1024) dtype=float32, numpy=
 array([[-0.00702935, -0.06676127, -0.04407291, ..., -0.01047852,
          0.06464547,  0.02131308],
        [ 0.03347615,  0.00835491, -0.04214556, ...,  0.03264932,
          0.00427552

In [13]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    # 把model当函数来用，实际是调用类的call方法
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [14]:
example_batch_predictions[0][0]

<tf.Tensor: shape=(65,), dtype=float32, numpy=
array([ 0.0113624 , -0.0326306 ,  0.04214111, -0.01659971, -0.01778518,
       -0.00654469, -0.03008696,  0.0028754 , -0.00301468, -0.03413392,
       -0.00217845,  0.04114346,  0.05130856, -0.02169739, -0.00398294,
       -0.00693167,  0.01020009,  0.04199206,  0.01656809,  0.00454826,
        0.0169168 , -0.01643466,  0.03392875, -0.02004773,  0.00881675,
       -0.01126692,  0.03157157, -0.00883924,  0.00421403, -0.03911354,
       -0.02010616, -0.03110015, -0.02959238, -0.00530336,  0.01399101,
       -0.04115472, -0.00500408,  0.04638671,  0.04460899,  0.00499319,
        0.00623027, -0.00924729, -0.02157588, -0.05315569,  0.00062216,
       -0.04730015, -0.01823638, -0.02424935, -0.03827185,  0.0378241 ,
       -0.00108806,  0.02002366,  0.03210833, -0.02465087, -0.00871665,
       -0.00246845,  0.01842871,  0.00327547,  0.02961155,  0.02823824,
        0.06209011, -0.01669264, -0.0037242 , -0.00405848, -0.00357904],
      dtype=floa

In [15]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 65), dtype=float32, numpy=
array([[ 0.0113624 , -0.0326306 ,  0.04214111, ..., -0.0037242 ,
        -0.00405848, -0.00357904],
       [-0.01187989, -0.05884358, -0.04518011, ...,  0.01021862,
         0.0156815 , -0.01187882],
       [ 0.04486758, -0.01728885, -0.00642166, ..., -0.05024748,
         0.01857883, -0.03882169],
       ...,
       [ 0.22144863,  0.04864826, -0.265059  , ..., -0.06042609,
        -0.01507821, -0.00308478],
       [-0.0110305 ,  0.11021354, -0.14840022, ...,  0.14240268,
         0.15840203,  0.06076766],
       [ 0.5676479 ,  0.24685271, -0.27520013, ...,  0.11362644,
        -0.09338273, -0.1906892 ]], dtype=float32)>

In [16]:
# random sampling.
# greedy, random.
# logits是计算分类任务之前，没有经过softmax的那个值就是logits
# tf.random.categorical从分类分布中抽取样本,随机为了每次写文章有差异
print(example_batch_predictions[0][0])
print('-'*100)
sample_indices = tf.random.categorical(
    logits = example_batch_predictions[0], num_samples = 1,seed=1)
print(sample_indices)  # 这里的维度是（100,1）
print('-'*100)
# (100, 1) -> (100, )  调用squeeze 去除1的维度，变为100的向量
sample_indices = tf.squeeze(sample_indices, axis = -1)
print(sample_indices)

tf.Tensor(
[ 0.0113624  -0.0326306   0.04214111 -0.01659971 -0.01778518 -0.00654469
 -0.03008696  0.0028754  -0.00301468 -0.03413392 -0.00217845  0.04114346
  0.05130856 -0.02169739 -0.00398294 -0.00693167  0.01020009  0.04199206
  0.01656809  0.00454826  0.0169168  -0.01643466  0.03392875 -0.02004773
  0.00881675 -0.01126692  0.03157157 -0.00883924  0.00421403 -0.03911354
 -0.02010616 -0.03110015 -0.02959238 -0.00530336  0.01399101 -0.04115472
 -0.00500408  0.04638671  0.04460899  0.00499319  0.00623027 -0.00924729
 -0.02157588 -0.05315569  0.00062216 -0.04730015 -0.01823638 -0.02424935
 -0.03827185  0.0378241  -0.00108806  0.02002366  0.03210833 -0.02465087
 -0.00871665 -0.00246845  0.01842871  0.00327547  0.02961155  0.02823824
  0.06209011 -0.01669264 -0.0037242  -0.00405848 -0.00357904], shape=(65,), dtype=float32)
----------------------------------------------------------------------------------------------------
tf.Tensor(
[[31]
 [63]
 [64]
 [23]
 [51]
 [15]
 [40]
 [ 1]
 [48]
 [

In [17]:
# 理解random.categorical，虽然是随机的，但是还是偏向于概率较大的值
# num_samples，意味着我们搞几次抽样
for i in tf.range(5):
    samples = tf.random.categorical([[4.0,2.0,2.0,2.0,1.0]], 3)
    tf.print(samples)

[[0 0 0]]
[[0 0 0]]
[[3 0 0]]
[[3 0 0]]
[[0 0 0]]


In [18]:
print("Input: ", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Output: ", repr("".join(idx2char[target_example_batch[0]])))
print()
print("Predictions: ", repr("".join(idx2char[sample_indices])))

Input:  'ly in both, if justice had her right.\n\nLORD ROSS:\nMy heart is great; but it must break with silence,'

Output:  'y in both, if justice had her right.\n\nLORD ROSS:\nMy heart is great; but it must break with silence,\n'

Predictions:  "SyzKmCb jzEfu'bf$h;orF,M.'cGCZy-FQrzfrSzpcgVwcDgTbPxrFWffttooQOTP:XrhLKP\nxL,,xFTjFAAjVK -lqoyM&mwQta"


In [19]:
# from_logits是否预期为对数张量。默认情况下，我们假设对概率分布进行编码
# logits表示网络的直接输出 。没经过sigmoid或者softmax的概率化。
# from_logits=False就表示把已经概率化了的输出，重新映射回原值。log（p/(1-p)）
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)

model.compile(optimizer = 'adam', loss = loss)
example_loss = loss(target_example_batch, example_batch_predictions)
print(example_loss.shape)
print(example_loss.numpy().mean())  #看下样例的loss

(64, 100)
4.1827135


In [20]:
# 定义一个文件夹，保存模型
output_dir = "./text_generation_checkpoints"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    # 只保存权重的值
    save_weights_only = True)

epochs = 100
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])


Epoch 1/100
172/172 [==============================] - 14s 69ms/step - loss: 2.7582
Epoch 2/100
172/172 [==============================] - 13s 64ms/step - loss: 2.0147
Epoch 3/100
172/172 [==============================] - 13s 66ms/step - loss: 1.8030
Epoch 4/100
172/172 [==============================] - 13s 64ms/step - loss: 1.6723
Epoch 5/100
172/172 [==============================] - 13s 63ms/step - loss: 1.5873
Epoch 6/100
172/172 [==============================] - 13s 64ms/step - loss: 1.5276
Epoch 7/100
172/172 [==============================] - 13s 65ms/step - loss: 1.4838
Epoch 8/100
172/172 [==============================] - 13s 68ms/step - loss: 1.4515
Epoch 9/100
172/172 [==============================] - 13s 63ms/step - loss: 1.4218
Epoch 10/100
172/172 [==============================] - 13s 69ms/step - loss: 1.3968
Epoch 11/100
172/172 [==============================] - 13s 64ms/step - loss: 1.3776
Epoch 12/100
172/172 [==============================] - 13s 68ms/step - lo

In [21]:
tf.train.latest_checkpoint(output_dir)

'./text_generation_checkpoints/ckpt_100'

In [22]:
output_dir = "./text_generation_checkpoints"
model2 = build_model(vocab_size,
                     embedding_dim,
                     rnn_units,
                     batch_size = 1)
model2.load_weights(tf.train.latest_checkpoint(output_dir))
# 1是一个样本，None是可以变长序列
# model2.build(tf.TensorShape([1, None]))

# 下面是文本生成的流程
# start ch sequence A, 
# A -> model -> b  A放入模型后得到b
# A.append(b) -> B
# B(Ab) -> model -> c
# B.append(c) -> C
# C(Abc) -> model -> ...
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (1, None, 1024)           1311744   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 1,395,009
Trainable params: 1,395,009
Non-trainable params: 0
_________________________________________________________________


# 文本生成重要部分

In [23]:
# 定义一个函数来实现上面的文本生成流程
def generate_text(model, start_string, num_generate = 1000):
    # 这一次输出的是1维的,把字母变为对应的id
    input_eval = [char2idx[ch] for ch in start_string]
    print(input_eval)
    # 做一个维度扩展
    input_eval = tf.expand_dims(input_eval, 0)
    print(input_eval)
    text_generated = []
    # 对model进行reset，连续调用的时候使用resets_states()
    model.reset_states()
    
    for _ in range(num_generate):
        # 1. model inference -> predictions
        # 2. sample -> ch -> text_generated.
        # 3. update input_eval
        
        # predictions : [batch_size, input_eval_len, vocab_size]
        predictions = model(input_eval)
#         print(predictions.shape)
        # squeeze消掉 batch_size，变为predictions : [input_eval_len, vocab_size]
        predictions = tf.squeeze(predictions, 0)
        # predicted_ids: [input_eval_len, 1]
        # a b c -> b c d
#         print(predictions)
        # 把predictions: [input_eval_len, vocab_size]维度数据变为 1个维度
        predicted_id = tf.random.categorical(
            predictions, num_samples = 1)[-1, 0].numpy()  # 65个logits得到是什么类别
#         print(predicted_id)
        # 得到预测id后，放入text_generated
        text_generated.append(idx2char[predicted_id])
        # 下面这是是我们原来的公式,为什么没有append作为新的输入,因为那样比较低效
        # s, x -> rnn -> s', y
        input_eval = tf.expand_dims([predicted_id], 0)
#         print(input_eval)
    return start_string + ''.join(text_generated)

new_text = generate_text(model2, "All: ")
print(new_text)


[13, 50, 50, 10, 1]
tf.Tensor([[13 50 50 10  1]], shape=(1, 5), dtype=int32)
All: what lies your numbla?

DUKE VINCENTIO:
Is she should seek it,
Pardon men
How it still boyst-blown.

QUEEN:
With all your holy bond
To his molefily.
What simples of twice
O, yet the eyest is face,
That, in the eyes:
Hear'sts hath wounds the watery unto the lords and plain.

MENENIUS:
Ay, were I think, Katian you?' arm, when he married.

MENENIUS:
Do not frown in hell,
Wave justice, sty wedve love,
But night; the loss ingrorance.

WARWICK:
O,
SP year to his royal-or
And put or crustice:
How far I see the trotis agrees,
Put on others, belly born
His stare of rarewing close Phants are sweet to do,
How many me annage: therefore lest have heart till both a
makety unto the subject lim
despite of the horsem'd: take your speed.
In fanding was worthbowns; at the thorns
To frantica fortunes
Do gleatent this be lord's own belmon can obseement of the notestoies to this place.

MENENIUS:
Is aw the castl.
Thus I shall 